In [26]:
import requests
import lxml
from bs4 import BeautifulSoup
import os
import pandas as pd

第一步 先找到要下载的论文。`2203.04114`就是论文的id，22 表示是22年的论文，03表示是3月的论文，后面的04114就是论文的编号。我们想得到这篇论文里面的所有公式，但是原始的PDF文档不能得到公式对应的`mathjax`语法
比如刚才看到的公式语法如下：
`L=-\sum_{i=1}^{10}y_i \log p_i`
对应的渲染结果就是：
$$
L=-\sum_{i=1}^{10}y_i \log p_i
$$

好在arxiv收录了它的latex源文件工程，因此可以从其中提取公式的mathjax表达。先下载回来看看长啥样：

使用了一个`get`请求`https://arxiv.org/e-print/2203.04114`，可以看到这个源文件和文件id是对应的，不放心的话可以多找几个来看看，像刚刚那种就没有latex文件 我们直接放弃。看看下载的文件。他应该是`.tar.gz`格式的文件我们重命名一下

可见，`\begin{equation}\bm{E}_\text{V} = f_V(\bm{I_\text{V}}).\end{equation}`间的内容就是公式，我们需要提取这部分的内容。重新整理下思路，首先得到paper的id后，下载文件并重命名->解压->找到`.tex`文件
-> 找到latex公式对应的数据->保存。一份论文大概有1Mb左右的大小，大量处理的时候记得及时清理数据。

Step1: 找到paper的id。

后面的参数，有一个skip和show，show应该是指每页展示的paper数，skip就是跳过多少个paper，还有能看到有些paper是没有源文件的。我们筛选那些有源文件的

In [27]:
url = 'https://arxiv.org/list/cs/2001?skip=0&show=100'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54'
}
req = requests.get(url,headers)

每个页最好别显示太多，不然加载和解析会很慢，下面获取有format的index。

In [28]:
soup = BeautifulSoup(req.text, 'lxml')


d:\python3.9\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [29]:
results = soup.find_all('a', title = 'Other formats' )
for each in results[:10]:
    print(each.get('href')[8:])

2001.00003
2001.00004
2001.00006
2001.00008
2001.00009
2001.00046
2001.00047
2001.00048
2001.00051
2001.00053


In [30]:
papers_id = [each.get('href')[8:] for each in results]

下面用一个表格记录一下这些合法的paper：

In [31]:
data = pd.DataFrame()
data['paper_id'] = papers_id

In [32]:
data.head()

,paper_id
0,2001.00003
1,2001.00004
2,2001.00006
3,2001.00008
4,2001.00009


拿到了id后可以下载回来

In [39]:
id = papers_id[0]
def download_tar(file_name, dir):
    '''
    下载 file 到 dir
    '''
    url_base = 'https://arxiv.org/e-print/'
    url = url_base + file_name
    file_name += '.tar.gz'
    file_path = os.path.join(dir, file_name)
    if os.path.exists(file_path):
        print("File {} Already Exisist".format(file_name))
        return file_path
    else:
        req = requests.get(url,headers=headers)
        if req.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(req.content)
            print('Finish Download: {}'.format(file_name))
            return file_path
        else:
            print('Download {} Failed.'.format(file_name))
            return None

In [40]:
tar_file = download_tar(id,'Downloads/')

File 2001.00003.tar.gz Already Exisist


这下载也太慢了，解压一下：

In [42]:
import tarfile
def un_tar(file):
    tar = tarfile.open(file)
    names = tar.getnames()
    if os.path.isdir(file + '_files'):
        pass
    else:
        os.mkdir(file + '_files')
        for name in names:
            tar.extract(name, file + '_files')
    return file + '_files'

untar_file = un_tar(tar_file)

下面就要找到其中的tex文件

In [45]:
import glob
def find_texs(untar_file):
    texs = glob.glob('{}/*.tex'.format(untar_file))
    return texs
texs = find_texs(untar_file)

下一步解析latex公式

In [49]:
import re
def parse_tex(file):
    rule = re.compile(r'\\begin{equation}(.*?)\\end{equation}',re.S)
    with open(file, 'rb') as f:
        content = f.read().decode('utf-8')
        content = content.replace('\n','')
        results = rule.findall(content)
    return results
    
formula = parse_tex(texs[0])

或者直接把一篇文章的所有tex文件中的公式保存起来。用正则筛选，多行匹配。

In [51]:
import re
def parse_texs(files):
    results = []
    rule = re.compile(r'\\begin{equation}(.*?)\\end{equation}',re.S)
    for file in files:
        with open(file, 'rb') as f:
            content = f.read().decode('utf-8')
            content = content.replace('\n','')
            result = rule.findall(content)
        results += result
    return {index:value for index, value in enumerate(results)}
    
formula = parse_texs(texs)

最后把已经提取好的文件删除省内存。要删除`tar`文件和解压后的`tar_files`文件夹。

In [52]:
import shutil
def clean_files(file):
    # 以id作为标识
    # 先删除文件夹
    shutil.rmtree(untar_file)
    os.remove(tar_file)
clean_files(id[0])


整个流程过完了，可以整合到一起了。哦对还要把资料保存到`DataFrame`里：

In [53]:
data.head()

,paper_id
0,2001.00003
1,2001.00004
2,2001.00006
3,2001.00008
4,2001.00009


In [59]:
for row in data.iterrows():
    data['formula'] = str(formula)
    break

In [60]:
data.head()

,paper_id,formula
0,2001.00003,{0: ' e(n) = \\cdot \\sum_{p \\in \\mathbb...
1,2001.00004,{0: ' e(n) = \\cdot \\sum_{p \\in \\mathbb...
2,2001.00006,{0: ' e(n) = \\cdot \\sum_{p \\in \\mathbb...
3,2001.00008,{0: ' e(n) = \\cdot \\sum_{p \\in \\mathbb...
4,2001.00009,{0: ' e(n) = \\cdot \\sum_{p \\in \\mathbb...
